This script is a snippet to make it clear how the Dragonfly FOV is extracted from the data.

The Dragonfly FOV depends on the distance and resolution/size of the data (because the script needs to know the pixel scale).  Give it a reference point that it will center the Dragonfly FOV on.

In [1]:
import numpy as np
import get_halpha_SB

pixscale =  {'50Mpc': 0.237/1000.*(1.+0.0115), '100Mpc': 0.477/1000.*(1.+0.0235),'200Mpc': 0.928/1000.*(1.+0.047) , '500Mpc': 2.178/1000.*(1.+0.12)} ### Mpc / arcsec (comoving)
x_angFOV = 3.*60.*60. # " 
y_angFOV = 2.*60.*60. # "  
x_FOV = {distance: pixscale[distance]*x_angFOV for distance in ['50Mpc','100Mpc','200Mpc','500Mpc']}  # cMpc
y_FOV = {distance: pixscale[distance]*y_angFOV for distance in ['50Mpc','100Mpc','200Mpc','500Mpc']}  # cMpc


In [2]:
x_center = 47.5
y_center = 12.